# Week #4
## Линейная регрессия
### Прогноз оклада по описанию вакансии

Линейные методы хорошо подходят для работы с разреженными данными — к таковым относятся, например, тексты. Это можно объяснить высокой скоростью обучения и небольшим количеством параметров, благодаря чему удается избежать переобучения.

Линейная регрессия имеет несколько разновидностей в зависимости от того, какой регуляризатор используется. Мы будем работать с гребневой регрессией, где применяется квадратичный, или L2-регуляризатор.

Для извлечения TF-IDF-признаков из текстов воспользуйтесь классом sklearn.feature_extraction.text.TfidfVectorizer.

Для предсказания целевой переменной мы будем использовать гребневую регрессию, которая реализована в классе sklearn.linear_model.Ridge.

Обратите внимание, что признаки LocationNormalized и ContractTime являются строковыми, и поэтому с ними нельзя работать напрямую. Такие нечисловые признаки с неупорядоченными значениями называют категориальными или номинальными. Типичный подход к их обработке — кодирование категориального признака с m возможными значениями с помощью m бинарных признаков. Каждый бинарный признак соответствует одному из возможных значений категориального признака и является индикатором того, что на данном объекте он принимает данное значение. Данный подход иногда называют one-hot-кодированием. Воспользуйтесь им, чтобы перекодировать признаки LocationNormalized и ContractTime. Он уже реализован в классе sklearn.feature_extraction.DictVectorizer. Пример использования:

from sklearn.feature_extraction import DictVectorizer

enc = DictVectorizer()

X_train_categ = enc.fit_transform(data_train[['LocationNormalized',         
                                              'ContractTime']].to_dict('records'))

X_test_categ = enc.transform(data_test[['LocationNormalized', 
                                        'ContractTime']].to_dict('records'))
                                        
                                        
Вам понадобится производить замену пропущенных значений на специальные строковые величины (например, 'nan'). Для этого подходит следующий код:

data_train['LocationNormalized'].fillna('nan', inplace=True)

data_train['ContractTime'].fillna('nan', inplace=True)   

In [34]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import Ridge
from scipy.sparse import hstack

In [12]:
# Загрузите данные об описаниях вакансий и соответствующих годовых зарплатах из файла 
# salary-train.csv (либо его заархивированную версию salary-train.zip).
data_train = pd.read_csv('data/salary-train.csv')
data_train.head(5)

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,International Sales Manager London ****k ****...,London,permanent,33000
1,An ideal opportunity for an individual that ha...,London,permanent,50000
2,Online Content and Brand Manager// Luxury Reta...,South East London,permanent,40000
3,A great local marketleader is seeking a perman...,Dereham,permanent,22500
4,Registered Nurse / RGN Nursing Home for Young...,Sutton Coldfield,NaN,20355


In [27]:
# Проведите предобработку:
# - Приведите тексты к нижнему регистру (text.lower()).
# - Замените все, кроме букв и цифр, на пробелы — это облегчит дальнейшее разделение текста 
#   на слова. Для такой замены в строке text подходит следующий вызов: 
#       re.sub('[^a-zA-Z0-9]', ' ', text). 
#   Также можно воспользоваться методом replace у DataFrame, чтобы сразу преобразовать 
#   все тексты:
#       train['FullDescription'] = 
#           train['FullDescription'].replace('[^a-zA-Z0-9]', ' ', regex = True)
# - Примените TfidfVectorizer для преобразования текстов в векторы признаков. 
#   Оставьте только те слова, которые встречаются хотя бы в 5 объектах (параметр min_df 
#   у TfidfVectorizer).
# - Замените пропуски в столбцах LocationNormalized и ContractTime на специальную строку 'nan'. 
#   Код для этого был приведен выше.
# - Примените DictVectorizer для получения one-hot-кодирования признаков LocationNormalized 
#   и ContractTime.
# - Объедините все полученные признаки в одну матрицу "объекты-признаки". Обратите внимание, 
#   что матрицы для текстов и категориальных признаков являются разреженными. Для объединения 
#   их столбцов нужно воспользоваться функцией scipy.sparse.hstack.
data_train['FullDescription'] = data_train['FullDescription'].str.lower().replace('[^a-z0-9]', ' ', regex=True)
tfidf_vectorizer = TfidfVectorizer(min_df=5)
X_train1 = tfidf_vectorizer.fit_transform(data_train['FullDescription'])

In [39]:
data_train['LocationNormalized'].fillna('nan', inplace=True)
data_train['ContractTime'].fillna('nan', inplace=True)
dict_vectorizer = DictVectorizer()
X_train2 = dict_vectorizer.fit_transform(
    data_train[['LocationNormalized', 'ContractTime']].to_dict('records')) 

X_train = hstack([X_train1, X_train2])
y_train = data_train['SalaryNormalized']

In [60]:
# Обучите гребневую регрессию с параметрами alpha=1 и random_state=241. 
# Целевая переменная записана в столбце SalaryNormalized.
method = Ridge(alpha=1, random_state=241)
a = method.fit(X_train, y_train)

In [66]:
# Постройте прогнозы для двух примеров из файла salary-test-mini.csv. 
# Значения полученных прогнозов являются ответом на задание. Укажите их через пробел.
data_test = pd.read_csv('data/salary-test-mini.csv')
X_test1 = tfidf_vectorizer.transform(data_test['FullDescription'])
data_train['LocationNormalized'].fillna('nan', inplace=True)
data_train['ContractTime'].fillna('nan', inplace=True)
X_test2 = dict_vectorizer.transform(
    data_test[['LocationNormalized', 'ContractTime']].to_dict('records'))
X_test = hstack([X_test1, X_test2])
y_test_predict = method.predict(X_test)
print ' '.join(str(y) for y in y_test_predict)

56555.6150015 37188.3244262


## Понижение размерности и метод главных компонент
### Составление фондового индекса

Метод главных компонент (principal component analysis, PCA) — это один из методов обучения без учителя, который позволяет сформировать новые признаки, являющиеся линейными комбинациями старых. При этом новые признаки строятся так, чтобы сохранить как можно больше дисперсии в данных. Иными словами, метод главных компонент понижает размерность данных оптимальным с точки зрения сохранения дисперсии способом.

Основным параметром метода главных компонент является количество новых признаков. Как и в большинстве методов машинного обучения, нет четких рекомендаций по поводу выбора значения этого параметров. Один из подходов — выбирать минимальное число компонент, при котором объясняется не менее определенной доли дисперсии (это означает, что в выборке сохраняется данная доля от исходной дисперсии).

В этом задании понадобится измерять схожесть двух наборов величин. Если имеется набор пар измерений (например, одна пара — предсказания двух классификаторов для одного и того же объекта), то охарактеризовать их зависимость друг от друга можно с помощью корреляции Пирсона. Она принимает значения от -1 до 1 и показывает, насколько данные величины линейно зависимы. Если корреляция равна -1 или 1, то величины линейно выражаются друг через друга. Если она равна нулю, то линейная зависимость между величинами отсутствует.

В этом задании мы будем работать с данными о стоимостях акций 30 крупнейших компаний США. На основе этих данных можно оценить состояние экономики, например, с помощью индекса Доу-Джонса. Со временем состав компаний, по которым строится индекс, меняется. Для набора данных был взят период с 23.09.2013 по 18.03.2015, в котором набор компаний был фиксирован (подробнее почитать о составе можно по ссылке из материалов).

Одним из существенных недостатков индекса Доу-Джонса является способ его вычисления — при подсчёте индекса цены входящих в него акций складываются, а потом делятся на поправочный коэффициент. В результате, даже если одна компания заметно меньше по капитализации, чем другая, но стоимость одной её акции выше, то она сильнее влияет на индекс. Даже большое процентное изменение цены относительно дешёвой акции может быть нивелировано незначительным в процентном отношении изменением цены более дорогой акции.

Метод главных компонент реализован в пакете scikit-learn в модуле decomposition в классе PCA. Основным параметром является количество компонент (n_components). Для обученного преобразования этот класс позволяет вычислять различные характеристики. Например, поле explained_variance_ratio_ содержит процент дисперсии, который объясняет каждая компонента. Поле components_ содержит информацию о том, какой вклад вносят признаки в компоненты. Чтобы применить обученное преобразование к данным, можно воспользоваться методом transform.

Для нахождения коэффициента корреляции Пирсона можно воспользоваться функцией corrcoef из пакета numpy.

Ссылки: 

https://en.wikipedia.org/wiki/Dow_Jones_Industrial_Average
https://en.wikipedia.org/wiki/Historical_components_of_the_Dow_Jones_Industrial_Average

In [82]:
from sklearn.decomposition import PCA

In [110]:
# Загрузите данные close_prices.csv. В этом файле приведены цены акций 30 компаний 
# на закрытии торгов за каждый день периода.
data_train = pd.read_csv('data/close_prices.csv', index_col='date')
data_train.head(5)

,AXP,BA,CAT,CSCO,CVX,DD,DIS,GE,GS,HD,...,PFE,PG,T,TRV,UNH,UTX,V,VZ,WMT,XOM
date,,,,,,,,,,,,,,,,,,,,,
2013-09-23,76.440002,117.510002,85.029999,24.270000,125.519997,59.409999,64.750000,24.280001,165.250000,75.910004,...,28.799999,79.279999,34.220001,86.379997,71.820000,109.419998,196.240005,47.980000,76.419998,87.750000
2013-09-24,76.070000,119.000000,85.110001,24.139999,124.489998,59.319997,64.320000,24.320000,162.970001,76.040001,...,28.709999,78.620003,34.090000,85.870003,72.320000,110.000000,193.339996,47.270000,75.750000,87.360001
2013-09-25,75.989998,118.510002,84.500000,24.430000,124.070000,59.319997,64.449997,24.230000,162.309998,75.519997,...,28.490000,77.720001,34.049999,85.980003,71.980003,109.260002,191.559998,46.950001,74.650002,87.139999
2013-09-26,76.320000,119.379997,84.199997,23.770000,123.489998,59.509996,65.239998,24.250000,162.289993,76.070000,...,28.520000,78.050003,34.230000,85.830002,72.160004,109.660004,193.559998,47.669998,74.620003,87.070000
2013-09-27,75.889999,118.739998,83.800003,23.330000,122.639999,59.009995,65.190002,24.049999,159.850006,75.959999,...,28.879999,77.209999,33.980000,85.410004,71.989998,109.360001,193.050003,47.000000,74.360001,86.900002


In [158]:
# На загруженных данных обучите преобразование PCA с числом компоненты равным 10. 
# Скольких компонент хватит, чтобы объяснить 90% дисперсии?
method = PCA(n_components=10)
data_train_new = method.fit_transform(data_train)
# print method.components_ # вклад признаков в главные компоненты
# print method.explained_variance_ratio_ # процент дисперсии, который объясняет каждая компонента

for i in range(method.explained_variance_ratio_.shape[0]):
    print (i+1), np.sum(method.explained_variance_ratio_[:i+1])

1 0.738971182715
2 0.849042872814
3 0.898993755584
4 0.927742953784
5 0.949897432663
6 0.969213199174
7 0.975961730737
8 0.982102640924
9 0.985308582609
10 0.988364692911


In [163]:
# Примените построенное преобразование к исходным данным и возьмите значения первой компоненты.
comp0 = method.transform(data_train)[:,0]
# comp0

In [207]:
# Загрузите информацию об индексе Доу-Джонса из файла djia_index.csv. 
# Чему равна корреляция Пирсона между первой компонентой и индексом Доу-Джонса?
dj = pd.read_csv('data/djia_index.csv', index_col='date')
pirson_coef = np.corrcoef(comp0, dj['^DJI'])
print pirson_coef

[[ 1.          0.90965222]
 [ 0.90965222  1.        ]]


In [208]:
# Какая компания имеет наибольший вес в первой компоненте? Укажите ее название с большой буквы.
idx_max = np.argmax(method.components_[0])
print data_train.columns.values
print 'Company with most weight:', data_train.columns.values[idx_max]
print 'Visa'

['AXP' 'BA' 'CAT' 'CSCO' 'CVX' 'DD' 'DIS' 'GE' 'GS' 'HD' 'IBM' 'INTC' 'JNJ'
 'JPM' 'KO' 'MCD' 'MMM' 'MRK' 'MSFT' 'NKE' 'PFE' 'PG' 'T' 'TRV' 'UNH' 'UTX'
 'V' 'VZ' 'WMT' 'XOM']
Company with most weight: V
Visa
